# CELL_PHONE data -- Feature Engineering

# Imports

In [8]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
spark = SparkSession.builder.config("spark.sql.shuffle.partitions", "200").config("spark.executor.memory", "8g").config("spark.executor.instances", 20).appName('3G_2week').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%matplotlib inline

In [12]:
df = spark.read.parquet('/user/ngwh3132/WORK/cancan_project/3G_2W/df_phone_data.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+------+---------+----------+-----------+-----+
|        id|             ts_list|            pos_list|nb_cdr|day_night|ie_entropy|pos_entropy|   rg|
+----------+--------------------+--------------------+------+---------+----------+-----------+-----+
|0055555732|[1970-01-01 19:12...|[[113.84944,22.78...|     1|    [1,0]|      null|       null|  0.0|
|0055555739|[1970-01-01 19:21...|[[114.06486,22.53...|     4|    [4,0]|      null|       null|  0.0|
|0055556253|[1970-01-01 10:53...|[[114.03306,22.54...|     3|    [3,0]|      null|       null|  0.0|
|0055556463|[1970-01-01 23:51...|[[114.042984,22.5...|     4|    [1,3]|      null|       0.41|3.566|
|0055556863|[1970-01-01 11:13...|[[114.0634,22.574...|   312|[183,129]|[164,0.65]|       0.06|0.162|
|0055557412|[1970-01-01 03:22...|[[114.05944,22.56...|     1|    [0,1]|      null|       null|  0.0|
|0055557807|[1970-01-01 19:59...|[[114.06347,22.53...|    43|   [43,0]|[156,0.97]|       0.

In [15]:
df = df.withColumn('ie_avg', df['ie_entropy'].getItem('ie_avg')).withColumn('ie_entropy', df['ie_entropy'].getItem('entropy')).withColumn('nb_event_day', df['day_night'].getItem('nb_event_day')).withColumn('nb_event_night', df['day_night'].getItem('nb_event_night'))
df = df.select('id', 'nb_cdr', 'nb_event_day', 'nb_event_night', 'ie_avg', 'ie_entropy', 'pos_entropy', 'rg')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- nb_cdr: integer (nullable = true)
 |-- nb_event_day: integer (nullable = true)
 |-- nb_event_night: integer (nullable = true)
 |-- ie_avg: integer (nullable = true)
 |-- ie_entropy: float (nullable = true)
 |-- pos_entropy: float (nullable = true)
 |-- rg: float (nullable = true)

In [18]:
df.dtypes

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('id', 'string'), ('nb_cdr', 'int'), ('nb_event_day', 'int'), ('nb_event_night', 'int'), ('ie_avg', 'int'), ('ie_entropy', 'float'), ('pos_entropy', 'float'), ('rg', 'float')]

Rajouter une colonne **percent_nocturnal**

In [20]:
df = df.withColumn('percent_nocturnal', ((F.col('nb_event_night') * 100) / F.col('nb_cdr')).cast('int'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------------+--------------+------+----------+-----------+-----+-----------------+
|        id|nb_cdr|nb_event_day|nb_event_night|ie_avg|ie_entropy|pos_entropy|   rg|percent_nocturnal|
+----------+------+------------+--------------+------+----------+-----------+-----+-----------------+
|0055555732|     1|           1|             0|  null|      null|       null|  0.0|                0|
|0055555739|     4|           4|             0|  null|      null|       null|  0.0|                0|
|0055556253|     3|           3|             0|  null|      null|       null|  0.0|                0|
|0055556463|     4|           1|             3|  null|      null|       0.41|3.566|               75|
|0055556863|   312|         183|           129|   164|      0.65|       0.06|0.162|               41|
|0055557412|     1|           0|             1|  null|      null|       null|  0.0|              100|
|0055557807|    43|          43|             0|   156|      0.97|       0.28| 0.33

Compter les **NaN** pour chaque colonne

In [22]:
df_count_nan = df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns])
df_count_nan = df_count_nan.toPandas().T
df_count_nan

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                        0
id                      0
nb_cdr                  0
nb_event_day            0
nb_event_night          0
ie_avg             115412
ie_entropy         115412
pos_entropy        145919
rg                      0
percent_nocturnal       0

In [24]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

414271

Supprimer les lignes avec des **NaN** partout pour les métriques: **'ie_avg', 'ie_entropy', 'pos_entropy'**

In [23]:
df_new = df.dropna(how='all', subset = ['ie_avg', 'ie_entropy', 'pos_entropy'])
df_new.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

329076

Le nombre de lignes supprimées est:

In [25]:
df.count() - df_new.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

85195

Nombre de **NaN** par colonne après la suppression des lignes inutiles

In [26]:
df_count_nan = df_new.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df_new.columns])
df_count_nan = df_count_nan.toPandas().T
df_count_nan

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                       0
id                     0
nb_cdr                 0
nb_event_day           0
nb_event_night         0
ie_avg             30217
ie_entropy         30217
pos_entropy        60724
rg                     0
percent_nocturnal      0

#### Enregistrer le nouveau df 

In [27]:
df_new.write.parquet("/user/ngwh3132/WORK/cancan_project/3G_2W/new_df_phone_data.parquet", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Data viz

**numeric features**: description

In [28]:
numeric_features = [t[0] for t in df_new.dtypes if (t[1] == 'int') | (t[1] == 'float')]
stats_df = df_new.select(numeric_features).describe().toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
stats_df[['nb_cdr', 'nb_event_day', 'nb_event_night', 'ie_avg', 'ie_entropy', 'pos_entropy', 'rg', 'percent_nocturnal']] = stats_df[['nb_cdr', 'nb_event_day', 'nb_event_night', 'ie_avg', 'ie_entropy', 'pos_entropy', 'rg', 'percent_nocturnal']].apply(pd.to_numeric, errors='coerce')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
stats_df.T

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                        0     1       2    3      4
summary             count  mean  stddev  min    max
nb_cdr             329076   115     182    1   8308
nb_event_day       329076    71     126    0   6304
nb_event_night     329076    44      83    0   7604
ie_avg             298859   479     916    1  17188
ie_entropy         298859     0       0    0      1
pos_entropy        268352     0       0    0      1
rg                 329076     0       1    0     18
percent_nocturnal  329076    36      35    0    100

### CDF

In [13]:
# def make_cdf(df, feature):
    
#     """ 
#         return pandas df  with unique values of the feature and the frequence 
#     """
#     # Frequence 
#     df_new = df.groupby(F.col(feature)).agg(F.count(feature).alias('frequence')).select(feature, 'frequence').orderBy(feature).toPandas()
    
#     # PDF
#     df_new['pdf'] = df_new['frequence'] / sum(df_new['frequence'])

#     # CDF
#     df_new['cdf'] = df_new['pdf'].cumsum()
    
#     return(df_new)

Construire la **CDF** pour chaque métrique numérique dans les données

In [36]:
# for feature in numeric_features:
#     feature = make_cdf(df_new, feature)
#     print('CDF for', feature)
#     print(feature)
#     print('----------------------')
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Violin plot** for each feature in the distribution

In [35]:
# for feature in numeric_features:
#     cdf_feature = make_cdf(df_new, feature)
#     sns.violinplot(feature, data=cdf_feature)
#     plt.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…